In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

### a. Load data

In [2]:
folder = '/Users/daiyiluo/Downloads/KQ095_210205' #'/Users/daiyiluo/Downloads/KQ086_210112' #'/Users/daiyiluo/Downloads/caiman_t/dan_CA1' #

# load behavior data
data = np.load(f'{folder}/S.npz')
spd = data['spd']
dis = data['dis']
lk = data['lk']

del data

spd.shape, dis.shape, lk.shape

((96593,), (96593,), (96593,))

In [3]:
spd=spd.squeeze()
dis=dis.squeeze()

In [3]:
# load neural activity data to be of size (#sample * #Neurons)
# # Daniel
# spks0 = np.load(f'{folder}/spk.npz')['spks'].T
# traces = np.load(f'{folder}/spk.npz')['traces'].T

# Fish
spks0 = np.load(f'{folder}/spk.npz')['spks']
data = loadmat(f'{folder}/dFishQ_raw.mat')
data.keys() # check the name of the variable

dict_keys(['__header__', '__version__', '__globals__', 'B'])

In [4]:
key = 'B' # 'B' for CA1 data, 'dFishQ_raw' for CA3 data
traces = data[key]
del data
print(f'Shape of calcium traces: {traces.shape}')

Shape of calcium traces: (96593, 219)


In [5]:
%matplotlib

Using matplotlib backend: MacOSX


In [9]:
spks = np.round(spks0*50)

In [11]:
plt.plot(traces[:,1],label='Trace')
# plt.plot(spks[:,1],label='Spike')
plt.plot(spks0[:,1]*10,label='Spike')
plt.legend()

In [37]:
plt.figure()
for i,f in enumerate([11,13,14]):
    t = traces[:,f]
    t1 = avgtrc[:,f]
    s = spks0[:,f]
    plt.plot(t/np.max(t)+i,'b')
    plt.plot(s/np.max(s)+i,'r')
    plt.plot(idx_dis,t1/np.max(t1)+i ,'k.')
# plt.legend()

In [38]:
i=9
t = traces[:,i]
t1 = avgtrc[:,i]
s = spks0[:,i]
plt.plot(t/np.max(t)+i,'b', label = 'Trace')
plt.plot(s/np.max(s)+i,'r', label = 'Spike')
plt.plot(idx_dis,t1/np.max(t1)+i ,'k.',label = 'Binned Trace')
plt.legend()

In [42]:
plt.xlim(91120,91310)

(91120.0, 91310.0)

### b. Sort into time bin

In [13]:
new_tb = 500  # 2Hz: 500ms
ratio = int(30*new_tb/1000)
ratio

15

In [14]:
# Sample speed
avgspd = np.convolve(spd, np.ones(ratio)/ratio, mode='valid')
avgspd = avgspd[::ratio]

# Sample position
idx_dis = np.arange(round(ratio/2), len(dis)-round(ratio/2)+1, ratio)
avgdis = dis[idx_dis]

avgspd.shape, avgdis.shape

((6439,), (6439,))

---------- Trace -----------

In [15]:
# Sample traces
avgtrc = []
for i in range(traces.shape[1]):
    t = np.convolve(traces[:,i], np.ones(ratio)/ratio, mode='valid')
    avgtrc.append(t[::ratio])    
avgtrc = (np.vstack(avgtrc)).T

avgtrc.shape

(6439, 219)

---------- Spike -----------

In [10]:
# Sample spks
avgspk = []
for i in range(spks.shape[1]):
    t = np.convolve(spks[:,i], np.ones(ratio)/ratio, mode='valid')
    avgspk.append(t[::ratio]*ratio)    
avgspk = (np.vstack(avgspk)).T

avgspk.shape

(1879, 99)

### c. Exclude immobile time bins 

In [11]:
spd_thr = 0.03 # 0.02 for 1s tb, 0.03 for 200/500 ms
pass_thr = np.where(avgspd>spd_thr)[0]
%matplotlib
plt.plot(avgspd)
plt.plot(avgdis)
# plt.plot(pass_thr,avgdis[pass_thr],'*')
plt.plot(pass_thr,avgspd[pass_thr],'*')
plt.plot([0,len(avgspd)], [spd_thr,spd_thr])

Using matplotlib backend: MacOSX


In [12]:
Distance = avgdis[avgspd>spd_thr]
idx = idx_dis[avgspd>spd_thr] # for licking data because it is not sorted into 500ms time bins
Distance.shape, idx.shape

((1414,), (1414,))

---------- Trace -----------

In [13]:
Trace = avgtrc[avgspd>spd_thr,:]
Trace.shape

(1414, 99)

---------- Spike -----------

In [14]:
Spike = avgspk[avgspd>spd_thr,:]
Spike.shape

(1414, 99)

In [15]:
# Some neurons don't fire during running period
Trace = np.delete(Trace,obj=np.where(np.sum(Spike,axis=0)==0),axis=1)
Spike = np.delete(Spike,obj=np.where(np.sum(Spike,axis=0)==0),axis=1)

Trace.shape, Spike.shape

((1414, 98), (1414, 98))

In [76]:
Spike[:,57].max()

41.0

In [78]:
# Show all spike data

for i in range(Spike.shape[1]):
    plt.plot(Spike[:,i]/np.max(Spike[:,i])+i)

### d. Sort into spatial bins and Locate laps

In [16]:
l=avgdis.max()/49
Distance0=np.round(Distance/l)*l
np.unique(Distance0).shape

(50,)

In [17]:
plt.plot(Distance)
plt.plot(Distance0)

In [18]:
# nbin = 50 # expected number of spatial bins
# n = np.floor(1/(avgdis.max()/nbin))
# Distance0 = np.round(Distance*n)/n # round the distance
print(f'Number of spatial bins: {np.unique(Distance0).shape}\nLength of data: {Distance0.shape}')

Number of spatial bins: (50,)
Length of data: (1414,)


In [20]:
Distance = Distance0

d = np.diff(Distance)
lap_end = np.where(d<-1.3)[0] # Locate the point where the laps end

plt.plot(Distance)
plt.plot(lap_end,Distance[lap_end],'*')

In [21]:
Distance = Distance[lap_end[0]:]
idx = idx[lap_end[0]:]
f=lap_end[0]
print(f'Length of data after excluded the incomplete first lap: {Distance.shape}\nWhere the first complete lap starts: {f}')

Length of data after excluded the incomplete first lap: (1391,)
Where the first complete lap starts: 23


---------- Trace -----------

In [22]:
Trace = Trace[f:]
lap_end = lap_end-lap_end[0]
np.savez(f'{folder}/timebin_{new_tb}.npz', Trace=Trace, Distance=Distance, idx=idx, lap_end=lap_end)

Trace.shape, Distance.shape, idx.shape

((1391, 98), (1391,), (1391,))

---------- Spike -----------

In [25]:
Spike = Spike[f:]
np.save(f'{folder}/timebin_{new_tb}_spks.npy', Spike)

Spike.shape, Distance.shape, idx.shape

((1391, 99), (1391,), (1391,))